###Bài 4

In [1]:
!pip install transformers -q

from google.colab import drive
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

from transformers import ResNetForImageClassification

drive.mount('/content/drive')

drive_zip_path = '/content/drive/My Drive/VinaFood21.zip'

!mkdir -p ./data
!unzip -q -o "{drive_zip_path}" -d ./data

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

DATA_DIR = './data/VinaFood21'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

Mounted at /content/drive
Using device: cuda


In [2]:
input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.Resize(input_size + 32),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'test': datasets.ImageFolder(TEST_DIR, data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=64, shuffle=True, num_workers=2),
    'test': DataLoader(image_datasets['test'], batch_size=64, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
NUM_CLASSES = len(class_names)

print(f"Số lớp (NUM_CLASSES): {NUM_CLASSES}")
print(f"Kích thước tập huấn luyện: {dataset_sizes['train']}")
print(f"Kích thước tập test: {dataset_sizes['test']}")

Số lớp (NUM_CLASSES): 21
Kích thước tập huấn luyện: 10044
Kích thước tập test: 6682


In [7]:
import warnings
warnings.filterwarnings('ignore', message="The secret 'HF_TOKEN' does not exist in your Colab secrets.")

class PretrainedResnet(nn.Module):
    def __init__(self, num_classes=21):
        super().__init__()

        basemodel = ResNetForImageClassification.from_pretrained(
            "microsoft/resnet-50",
            ignore_mismatched_sizes=True
        )

        self.resnet = basemodel.resnet

        self.classifier = nn.Linear(in_features=2048, out_features=num_classes, bias=True)

    def forward(self, images: torch.Tensor):
        features = self.resnet(images).pooler_output
        features = features.squeeze(-1).squeeze(-1)
        logits = self.classifier(features)

        return logits

model = PretrainedResnet(num_classes=NUM_CLASSES).to(DEVICE)

In [5]:
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

for epoch in range(10):

    model.train()
    running_loss = 0.0

    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / dataset_sizes['train']

    print(f'Epoch [{epoch+1}/{10}], Loss: {epoch_loss:.4f}')

    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    scheduler.step()

    print(f"Learning Rate hiện tại: {scheduler.get_last_lr()[0]:.5f}")

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

print(f"Độ chính xác (Accuracy): {accuracy * 100:.2f}%")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"F1-score (Macro): {f1:.4f}")

Epoch [1/10], Loss: 0.6963
Learning Rate hiện tại: 0.00100
Epoch [2/10], Loss: 0.5439
Learning Rate hiện tại: 0.00100
Epoch [3/10], Loss: 0.4663
Learning Rate hiện tại: 0.00100
Epoch [4/10], Loss: 0.4228
Learning Rate hiện tại: 0.00010
Epoch [5/10], Loss: 0.3089
Learning Rate hiện tại: 0.00010
Epoch [6/10], Loss: 0.2574
Learning Rate hiện tại: 0.00010
Epoch [7/10], Loss: 0.2438
Learning Rate hiện tại: 0.00010
Epoch [8/10], Loss: 0.2342
Learning Rate hiện tại: 0.00001
Epoch [9/10], Loss: 0.2303
Learning Rate hiện tại: 0.00001
Epoch [10/10], Loss: 0.2195
Learning Rate hiện tại: 0.00001
Độ chính xác (Accuracy): 89.76%
Precision (Macro): 0.8965
Recall (Macro): 0.8999
F1-score (Macro): 0.8973
